In [5]:
import torch
# model = torch.load(
#     "/mnt/haojun/itpsea4data/mmseg/work_dirs/20221018_vitb16_attn4_cosine_160k_bs16_coco171_in130_seed0.1_coseg0.4/iter_160000.pth",
#     map_location="cpu"
# )
model2 = torch.load(
    "/mnt/haojun/itpsea4data/mmseg/work_dirs/vitb16_attn4_cosine_160k_bs16_coco171_in130_seed0.1_coseg0.2_mean/iter_100.pth",
    map_location="cpu"
)
model2["state_dict"]["decode_head.ptr"]

tensor([0, 1, 2, 0, 1, 3, 0, 0, 1, 0, 0, 0, 1, 1, 0, 3, 0, 0, 1, 2, 1, 2, 0, 5,
        0, 0, 5, 1, 0, 1, 0, 0, 0, 4, 0, 2, 0, 0, 0, 3, 0, 1, 0, 0, 0, 0, 0, 3,
        0, 0, 0, 0, 0, 0, 2, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 3, 3, 0, 0, 0, 3,
        0, 2, 3, 0, 1, 0, 1, 0, 1, 0, 2, 0, 0, 2, 1, 0, 3, 2, 0, 5, 3, 0, 1, 0,
        0, 3, 1, 1, 0, 2, 2, 5, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 2, 1, 3,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 4, 0, 0, 0, 0, 2, 1, 3, 1, 0, 0, 0,
        0, 2, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 2, 0, 0, 0, 1, 0, 5, 2, 2, 0, 3, 1,
        0, 2, 4, 1, 0, 0, 2, 2, 0, 0, 1, 0, 0, 2, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0,
        1, 1, 0, 0, 2, 3, 2, 1, 2, 1, 0, 0, 1, 0, 4, 3, 0, 0, 2, 0, 0, 0, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 1, 0, 1, 3, 1])

In [ ]:
for k in model2["state_dict"]:
    if "queue" in k:
        ptr = f"decode_head.ptr{k.lstrip('decode_head.queue')}"
        ptr = model2["state_dict"][ptr]
        queue = model2["state_dict"][k]
        queue = queue / queue.norm(dim=-1, keepdim=True)
        cos = queue @ queue.T
        mask = ~(torch.eye(len(cos)).bool())
        cos = cos[mask]
        print(k, cos.mean(), cos.max(), cos.min(), ptr)

In [12]:
from glob import glob

files = glob("./embeddings/*.pth")

In [14]:
file_pairs = {}
for f in files:
    img_cls = f.split("/")[-1].split("_")[1]
    if img_cls not in file_pairs:
        file_pairs[img_cls] = []
    file_pairs[img_cls].append(f)

In [25]:
a = torch.load(file_pairs["n03085602"][0], map_location="cpu").reshape(768, -1).permute(1,0)
b = torch.load(file_pairs["n04446276"][0], map_location="cpu").reshape(768, -1).permute(1,0)
a.shape, b.shape

(torch.Size([520, 768]), torch.Size([540, 768]))

In [26]:
cos = a @ b.T
print(cos.mean(), cos.std(), cos.max(), cos.min())

tensor(0.1687) tensor(0.1444) tensor(0.7368) tensor(-0.1785)


In [31]:
for img_cls in file_pairs:
    fs = file_pairs[img_cls]
    e1 = torch.load(fs[0], map_location="cpu").reshape(768, -1).permute(1,0)
    e2 = torch.load(fs[1], map_location="cpu").reshape(768, -1).permute(1,0)
    e1 = e1.mean(dim=0)
    e2 = e2.mean(dim=0)
    e1 = e1 / e1.norm(dim=-1, keepdim=True)
    e2 = e2 / e2.norm(dim=-1, keepdim=True)
    cos = e1 @ e2.T
    print(cos.mean(), cos.std(), cos.max(), cos.min())
    break

tensor(0.6521) tensor(nan) tensor(0.6521) tensor(0.6521)
